In [50]:
import json
import os
train_data_path = 'data/train/2025-04-11'
skip_files = ['stopwords']

def get_files(path: str) -> list:
    entries = os.listdir(path)
    return [os.path.join(path, entry) for entry in entries if os.path.isfile(os.path.join(path, entry))]

train_file_paths = get_files(train_data_path)
print(train_file_paths)

['data/train/2025-04-11\\1-相亲闪婚后，老公竟是千亿财阀.txt', 'data/train/2025-04-11\\10-腹黑娇妻：厉少，人设崩坏中.txt', 'data/train/2025-04-11\\100 小夫人一身反骨，大叔乖乖臣服.txt', 'data/train/2025-04-11\\101 墨爷的闪婚小甜妻.txt', 'data/train/2025-04-11\\102 流年不负你情深.txt', 'data/train/2025-04-11\\13-帝少甜婚令：娇妻，乖一点.txt', 'data/train/2025-04-11\\14-闪婚厚爱：病娇老公竟是千亿大佬.txt', 'data/train/2025-04-11\\17-闪婚三年不见面，再遇我是他秘书.txt', 'data/train/2025-04-11\\一夜惊喜：超级总裁喜当爹.txt', 'data/train/2025-04-11\\一胎六宝：总裁爸比超凶猛.txt', 'data/train/2025-04-11\\一胎双宝：季总你出局了.txt', 'data/train/2025-04-11\\七年之痒，前夫不知道我重生了.txt', 'data/train/2025-04-11\\七爷的罪妻.txt', 'data/train/2025-04-11\\三宝联萌 大佬妈咪拽又飒.txt', 'data/train/2025-04-11\\三宝联萌：大佬妈咪拽又飒.txt', 'data/train/2025-04-11\\三年恋情曝光，我成了白月光的替身.txt', 'data/train/2025-04-11\\不受宠？无所谓，杀疯豪门万人跪 (1~20章)-总裁、多重身份、傲慢霸气、复仇、男-有魅力的角色、浪漫、甜蜜、言情、大女主、甜宠.txt', 'data/train/2025-04-11\\不当舔狗后，肥妻她美又飒！ (1~20章)-多重身份、离婚、豪门霸总、真假千金、追妻火葬场、破镜重圆、甜宠、打脸.txt', 'data/train/2025-04-11\\乔秘书离职结婚，陆总醋疯了.txt', 'data/train/2025-04-11\\九零团宠三岁半.txt', 'data/train/2025-04-1

In [51]:
from src.util.read_file import read_file

encoding = 'utf-8'
train_novels = list[tuple]()

for file_path in train_file_paths:
    try:
        train_novels.append((file_path, read_file(file_path, encoding)))
    except Exception:
        print('Decode Error ', file_path)

with open('data/stopwords', encoding='utf-8') as _f:
    stopwords = _f.readlines()
    stopwords = [x.replace('\n', '') for x in stopwords]
    
if len(train_novels) > 0:
    print(str(train_novels[0])[:512], '...')
print(stopwords[:128])

Decode Error  data/train/2025-04-11\七爷的罪妻.txt
Decode Error  data/train/2025-04-11\三宝联萌 大佬妈咪拽又飒.txt
Decode Error  data/train/2025-04-11\大佬你的小娇娇甜翻了.txt
Decode Error  data/train/2025-04-11\星河灿烂皆因你.txt
Decode Error  data/train/2025-04-11\暖婚急诊：商医生，来盖个章！.txt
('data/train/2025-04-11\\1-相亲闪婚后，老公竟是千亿财阀.txt', '\n第1章 来闪婚还摆架子\n书名： 相亲闪婚后，老公竟是千亿财阀 作者： 一板奶片 本章字数： 2164 更新时间： 2023-01-18 21:15:55\n\n“清溪，今天晚上你在外面吃个饭再回来了，没钱舅舅给你，你舅妈在发疯呢……”\n\n电话那头舅舅的话还没说完，又紧跟着传来舅妈尖锐的声音。\n\n“我发疯？你那贱种侄女今年26岁还在读书，不嫁人在我家吃我的用我的，还说我发疯？！现在公司遇到这么大的危机，我不管，你必须把她介绍给……”\n\n余清溪刚准备开口，电话匆忙地挂断了。\n\n她无声地叹了口气，看着自己刚发的奖学金，一份没动给舅舅转了过去。\n\n这样的情况，基本上每个星期都会出现。\n\n自从她妈妈去世后，她被舅舅接回家。因为舅妈不喜欢她，她只能在这个家小心翼翼地生活着，直到考上了文物修复学的研究生，她才将前些年欠舅舅的学费用奖学金还干净了。\n\n本想着，这个家无论如何她都得搬走了，苦于舅舅一直不同意，舅妈也因此一直跟他吵架。\n\n她唯一能想到能够正大光明搬走，舅舅不会担心的借口，就是结婚 ...
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '--', '.', '..', '...', '......', '...................', './', '.一', '.数', '.日', '/', '//', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '://', '::', ';',

In [ ]:
from src.util.preprocess import split_words

train_seg_lists = []

# with open('data/seg.json', 'r', encoding='utf-8') as _f:
#     train_seg_lists = json.load(_f)
    
if len(train_seg_lists) != len(train_novels):
    for novel in train_novels[len(train_seg_lists):]:
        train_seg_lists.append((novel[0], split_words(novel[1], stopwords)))
        with open('data/seg.json', mode='w', encoding='utf-8') as _f:
            json.dump(train_seg_lists, _f, ensure_ascii=False)
    
print(train_seg_lists[0])

['摆架子', '裴奶奶', '上帝', '裴奶奶', '裴', '裴', '蹙眉头', '衍声色', '裴奶奶', '户口本', '户口本', '王总', '裴奶奶', '裴奶', '威逼利', '裴', '裴', '裴', '查过余', '裴', '吃吃饭', '余雯雯', '王家能', '王', '王', '王', '张', '王', '黄花大', '王总', '王', '老王', '王', '王', '王', '王', '王泽辉', '王泽辉', '王泽辉', '王泽辉', '王泽辉', '王泽辉', '王总', '王泽辉', '张牙舞爪', '王泽辉', '王泽辉', '王泽辉', '王泽辉', '王泽辉', '王泽辉', '裴', '王总', '暗示余', '江宇飞', '江宇飞', '林逸刚', '衍正', '裴', '裴', '张大', '林逸险', '裴爷爷', '林老爷子', '林老爷子', '林老爷子', '林老爷子']
['郝泽一', '郝泽气', '陆绾站', '苏', '陆婷婷', '王叔', '陆婷婷', '陆绾阖', '厉溟庭', '简历递', '迈巴赫', '厉溟庭', '阿林', '谢什么', '景林嘴', '歪头', '张峰', '景林无奈', '陆婷婷', '张导', '陆婷婷', '陆婷婷', '乔琨自幼', '厉溟庭', '乔琨愈', '王老五', '万里挑一', '陆', '陆', '张纸', '陆绾气']
['叶星语', '叶星语', '叶星语', '叶星语', '哗啦啦', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '陈嫂', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '谢青岑', '叶星语', '谢青岑', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '叶星语', '苏颜颜', '谢青岑', '谢青岑', '谢青岑', '谢青岑', '谢青岑', '叶星语', '叶星语', '叶星语', '苏颜颜', '叶星语', '白月光', '叶星语', '苏颜颜', '苏颜颜',

In [ ]:
train_documents = [x[1] for x in train_seg_lists]

In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(train_documents)
corpus = [dictionary.doc2bow(x) for x in train_documents]

print(corpus[:16])

In [ ]:
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

'''
Coherence=-0.018902030597915315, 2
Coherence=-1.0267733540287474, 3
Coherence=-4.1636033632647225, 4
Coherence=-4.119260146364487, 5
Coherence=-2.4559163650898, 6
Coherence=-2.1517339385259815, 7
Coherence=-2.462351387199522, 8
Coherence=-2.789655213062693, 9
Coherence=-2.213968382954225, 10
Coherence=-1.6386109139511134, 11
Coherence=-1.7469059597627445, 12
Coherence=-2.535528691204758, 13
Coherence=-1.529094511504894, 14
Coherence=-2.0892724235686675, 15
'''

num_topics = 4

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=64,
    random_state=32
)

cm = CoherenceModel(model=lda_model, texts=train_documents, coherence='u_mass')
coherence = cm.get_coherence()
print(f"Coherence={coherence}")
pyLDAvis.display(gensimvis.prepare(lda_model, corpus, dictionary))

In [ ]:
# save model
lda_model.save('model/lda/lda_model')